In [1]:
import os
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optimizers
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms

In [2]:
class DNN(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim):
        super().__init__()
        self.l1=nn.Linear(input_dim,hidden_dim)
        self.a1=nn.ReLU()
        self.l2=nn.Linear(hidden_dim,hidden_dim)
        self.a2=nn.ReLU()
        self.l3=nn.Linear(hidden_dim,hidden_dim)
        self.a3=nn.ReLU()
        self.l4=nn.Linear(hidden_dim,output_dim)
        
        self.layers=[self.l1,self.a1,self.l2,self.a2,self.l3,self.a3,self.l4]
    
    def forward(self,x):
        for layer in self.layers:
            x=layer(x)
        return x        

In [3]:
root=os.path.join('~','.torch','mnist')
transform=transforms.Compose([transforms.ToTensor(),lambda x: x.view(-1)])

mnist_train=datasets.MNIST(root=root,download=True,train=True,transform=transform)
mnist_test=datasets.MNIST(root=root,download=True,train=False,transform=transform)

train_dataloader=DataLoader(mnist_train,batch_size=100,shuffle=True)
test_dataloader=DataLoader(mnist_test,batch_size=100,shuffle=False)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=DNN(784,200,10).to(device)

In [7]:
criterion=nn.CrossEntropyLoss()
optimizer=optimizers.SGD(model.parameters(),lr=0.01)

def compute_loss(t,y):
    return criterion(y,t)

def train_step(x,t):
    model.train()
    preds=model(x)
    loss=compute_loss(t,preds)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss,preds

epochs=30
for epoch in range(epochs):
    train_loss=0.
    train_acc=0.
    for (x,t) in train_dataloader:
        x,t=x.to(device),t.to(device)
        loss,preds=train_step(x,t)
        train_loss+=loss.item()
        train_acc+=accuracy_score(t.tolist(),preds.argmax(dim=-1).tolist())
    train_loss/=len(train_dataloader)
    train_acc/=len(train_dataloader)
        
    print('epoch: {}, loss: {:.3f}, acc: {:.3f}'.format(epoch+1,train_loss,train_acc))

epoch: 1, loss: 2.275, acc: 0.287
epoch: 2, loss: 1.788, acc: 0.542
epoch: 3, loss: 0.732, acc: 0.793
epoch: 4, loss: 0.483, acc: 0.861
epoch: 5, loss: 0.405, acc: 0.883
epoch: 6, loss: 0.362, acc: 0.895
epoch: 7, loss: 0.332, acc: 0.904
epoch: 8, loss: 0.309, acc: 0.911
epoch: 9, loss: 0.289, acc: 0.916
epoch: 10, loss: 0.271, acc: 0.922
epoch: 11, loss: 0.255, acc: 0.927
epoch: 12, loss: 0.239, acc: 0.931
epoch: 13, loss: 0.226, acc: 0.935
epoch: 14, loss: 0.212, acc: 0.939
epoch: 15, loss: 0.201, acc: 0.943
epoch: 16, loss: 0.191, acc: 0.946
epoch: 17, loss: 0.181, acc: 0.949
epoch: 18, loss: 0.171, acc: 0.951
epoch: 19, loss: 0.163, acc: 0.954
epoch: 20, loss: 0.156, acc: 0.955
epoch: 21, loss: 0.149, acc: 0.958
epoch: 22, loss: 0.142, acc: 0.959
epoch: 23, loss: 0.136, acc: 0.961
epoch: 24, loss: 0.131, acc: 0.963
epoch: 25, loss: 0.126, acc: 0.965
epoch: 26, loss: 0.120, acc: 0.966
epoch: 27, loss: 0.116, acc: 0.967
epoch: 28, loss: 0.112, acc: 0.968
epoch: 29, loss: 0.107, acc: 

In [8]:
def test_step(x,t):
    model.eval()
    preds=model(x)
    loss=criterion(preds,t)
    return loss,preds

test_loss=0.
test_acc=0.

for (x,t) in test_dataloader:
    x,t=x.to(device),t.to(device)
    loss,preds=test_step(x,t)
    test_loss+=loss.item()
    test_acc+=accuracy_score(t.tolist(),preds.argmax(dim=-1).tolist())
test_loss/=len(test_dataloader)
test_acc/=len(test_dataloader)
print('test_loss: {:.3f}, test_acc: {:.3f}'.format(test_loss,test_acc))

test_loss: 0.117, test_acc: 0.964
